In [2]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import openpyxl
import akshare as ak
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 5000
pd.options.display.float_format = lambda x: '%.5f' % x
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='svg'
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime
import pymysql

In [3]:
DB_CONFIG = {
    "host": "192.168.133.132",  # Ubuntu的IP
    "user": "root",
    "password": "Zyice001347",    
    "charset": "utf8mb4",
}
# 初始化数据库连接
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# 1. 创建数据库（如果不存在）
cursor.execute("CREATE DATABASE IF NOT EXISTS a_shares CHARACTER SET utf8mb4;")
conn.select_db("a_shares")

def get_stock_codes():
 
    try:
        df=ak.stock_zh_a_spot()[['代码','名称']].rename(columns={'代码':'code','名称':'name'})
        #上证代码（包含科创板）
        code_sh=df
        code_sh=code_sh[code_sh['code'].str.startswith('sh')]
        code_sh=code_sh[(~code_sh['name'].str.startswith('ST'))&(~code_sh['name'].str.startswith('*ST'))]
        code_sh['code']=code_sh['code'].str[2:]

        #深证代码（包含创业板）
        code_sz=df
        code_sz=code_sz[code_sz['code'].str.startswith('sz')]
        code_sz=code_sz[(~code_sz['name'].str.startswith('ST'))&(~code_sz['name'].str.startswith('*ST'))]
        code_sz['code']=code_sz['code'].str[2:]

        #北证代码
        code_bj=df
        code_bj=code_bj[code_bj['code'].str.startswith('bj')]
        code_bj=code_bj[(~code_bj['name'].str.startswith('ST'))&(~code_bj['name'].str.startswith('*ST'))]
        code_bj['code']=code_bj['code'].str[2:]

        #A股所有代码
        code=pd.concat([code_bj,code_sh,code_sz],ignore_index=True).drop_duplicates().sort_values(by='code',ascending=True).reset_index(drop=True)
        return code['code'].tolist()
    except Exception as e:
        print(f"获取股票列表失败: {e}")
        return []


1

In [4]:
def fetch_and_save_stock(code):
    """爬取单只股票数据，仅当有新数据（len>0）时打印成功信息"""
    try:

        df = ak.stock_zh_a_hist(symbol=code, adjust="hfq")

        # 数据预处理
        df["日期"] = pd.to_datetime(df["日期"]).dt.date
        df.dropna(inplace=True)

        # 表名使用纯数字代码
        table_name = code

        # 判断表是否存在，不存在则创建
        cursor.execute(f"SHOW TABLES LIKE '{table_name}';")
        table_exists = cursor.fetchone() is not None

        if not table_exists:
            create_table_sql = f"""
            CREATE TABLE `{table_name}` (
                `日期` DATE NOT NULL PRIMARY KEY,
                `开盘` FLOAT,
                `收盘` FLOAT,
                `最高` FLOAT,
                `最低` FLOAT,
                `成交量` BIGINT,
                `成交额` BIGINT,
                `振幅` FLOAT,
                `涨跌幅` FLOAT,
                `涨跌额` FLOAT,
                `换手率` FLOAT
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
            """
            cursor.execute(create_table_sql)
            # 新表创建后，所有数据都是新的，无需筛选
            new_data_count = len(df)
        else:
            # 增量更新：只保留新数据
            cursor.execute(f"SELECT MAX(`日期`) FROM `{table_name}`;")
            max_date = cursor.fetchone()[0]
            if max_date:
                df = df[df["日期"] > max_date]
            new_data_count = len(df)
            if new_data_count == 0:
                # 无新数据，直接跳过（不打印成功信息）
                return

        # 批量插入新数据
        insert_sql = f"""
        INSERT IGNORE INTO `{table_name}` 
        (日期, 开盘, 收盘, 最高, 最低, 成交量, 成交额, 振幅, 涨跌幅, 涨跌额, 换手率)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        data = [tuple(row) for row in df[["日期", "开盘", "收盘", "最高", "最低", 
                                         "成交量", "成交额", "振幅", "涨跌幅", "涨跌额", "换手率"]].values]
        cursor.executemany(insert_sql, data)
        conn.commit()

        # 仅当有新数据（len>0）时打印成功信息
        if new_data_count > 0:
            print(f"股票 {code} 处理成功，新增 {new_data_count} 条数据")

    except Exception as e:
        print(f"股票 {code} 处理失败: {str(e)}")
        conn.rollback()

In [ ]:
stock_codes = get_stock_codes()
for code in stock_codes:
    fetch_and_save_stock(code)

cursor.close()
conn.close()

Please wait for a moment:   0%|          | 0/68 [00:00<?, ?it/s]

股票 000001 处理成功，新增 8201 条数据
股票 000002 处理成功，新增 8216 条数据
股票 000006 处理成功，新增 7886 条数据
股票 000007 处理成功，新增 7383 条数据
股票 000008 处理成功，新增 7693 条数据
股票 000009 处理成功，新增 8112 条数据
股票 000010 处理成功，新增 6210 条数据
股票 000011 处理成功，新增 7893 条数据
股票 000012 处理成功，新增 8017 条数据
股票 000014 处理成功，新增 7903 条数据
股票 000016 处理成功，新增 7930 条数据
股票 000017 处理成功，新增 6981 条数据
股票 000019 处理成功，新增 7587 条数据
股票 000020 处理成功，新增 7561 条数据
股票 000021 处理成功，新增 7530 条数据
股票 000025 处理成功，新增 7665 条数据
股票 000026 处理成功，新增 7707 条数据
股票 000027 处理成功，新增 7605 条数据
股票 000028 处理成功，新增 7574 条数据
股票 000029 处理成功，新增 6657 条数据
股票 000030 处理成功，新增 6872 条数据
股票 000031 处理成功，新增 7422 条数据
股票 000032 处理成功，新增 7376 条数据
股票 000034 处理成功，新增 6815 条数据
股票 000035 处理成功，新增 6472 条数据
股票 000036 处理成功，新增 7328 条数据
股票 000037 处理成功，新增 7156 条数据
股票 000039 处理成功，新增 7484 条数据
股票 000042 处理成功，新增 7188 条数据
股票 000045 处理成功，新增 7246 条数据
股票 000048 处理成功，新增 7203 条数据
股票 000049 处理成功，新增 7196 条数据
股票 000050 处理成功，新增 7080 条数据
股票 000055 处理成功，新增 7020 条数据
股票 000056 处理成功，新增 6743 条数据
股票 000058 处理成功，新增 6399 条数据
股票 000059 处理成功，新增 6719 条数据
股

股票 000869 处理成功，新增 5942 条数据
股票 000875 处理成功，新增 5326 条数据
股票 000876 处理成功，新增 6392 条数据
股票 000877 处理成功，新增 6314 条数据
股票 000878 处理成功，新增 6364 条数据
股票 000880 处理成功，新增 6330 条数据
股票 000881 处理成功，新增 6290 条数据
股票 000882 处理成功，新增 6367 条数据
股票 000883 处理成功，新增 6309 条数据
股票 000885 处理成功，新增 5883 条数据
股票 000886 处理成功，新增 6580 条数据
股票 000887 处理成功，新增 6087 条数据
股票 000888 处理成功，新增 6641 条数据
股票 000889 处理成功，新增 6195 条数据
股票 000890 处理成功，新增 6187 条数据
股票 000892 处理成功，新增 5633 条数据
股票 000893 处理成功，新增 5997 条数据
股票 000895 处理成功，新增 5882 条数据
股票 000897 处理成功，新增 6244 条数据
股票 000898 处理成功，新增 6610 条数据
股票 000899 处理成功，新增 6567 条数据
股票 000900 处理成功，新增 6164 条数据
股票 000901 处理成功，新增 6147 条数据
股票 000902 处理成功，新增 6145 条数据
股票 000905 处理成功，新增 6074 条数据
股票 000906 处理成功，新增 6083 条数据
股票 000910 处理成功，新增 6231 条数据
股票 000911 处理成功，新增 6202 条数据
股票 000912 处理成功，新增 5844 条数据
股票 000913 处理成功，新增 6239 条数据
股票 000915 处理成功，新增 6282 条数据
股票 000917 处理成功，新增 5988 条数据
股票 000919 处理成功，新增 6179 条数据
股票 000920 处理成功，新增 5909 条数据
股票 000921 处理成功，新增 5902 条数据
股票 000922 处理成功，新增 6091 条数据
股票 000923 处理成功，新增 6099 条数据
股